# Whatlies - Word embedding representations

In [2]:
%%capture
!pip install -Uqq whatlies[all]

In [7]:
import whatlies

from whatlies import Embedding

print("whatlies", whatlies.__version__)

whatlies 0.5.10
